In [486]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sqlite3
import re


In [487]:

# Read the CSV files into DataFrames
freight_df = pd.read_csv('../raw_data/Major_Freight_Users.csv')
traffic_df = pd.read_csv('../raw_data/Major_Traffic_Generators.csv')

In [488]:
#show the first few rows of the DataFrame
freight_df.head()

,OBJECTID,KBIF ID,Facility Name,Facility Street Address,City Name,State,Zip Code,County Name,Area Development District,NAICS Code,...,End Active Date,GlobalID,last_edited_date,created_user,created_date,last_edited_user,MP,RT_NUMBER,x,y
0,1,1522,Corning Incorporated,680 East Office St,Harrodsburg,KY,40330-1300,Mercer,BGADD,327211,...,NaN,e4f57a55-65d1-4385-a45f-77e50ec6e2df,6/30/2021 12:00:00 AM,NaN,NaN,EMILY.BARTEE,NaN,NaN,-9.443214e+06,4.545184e+06
1,2,4164,"FIAT Products, LLC",706 Highway 3057,Somerset,KY,42501-3302,Pulaski,LCADD,326191,...,NaN,32c75322-da75-4ef2-8676-2e7bd48ecd03,6/28/2024 12:00:00 AM,NaN,NaN,EMILY.BARTEE,NaN,NaN,-9.417573e+06,4.446790e+06
2,3,2169,Omico Plastics Inc,2025 Ragu Drive,Owensboro,KY,42303-,Daviess,GRADD,326199,...,NaN,52970649-d6fc-49d3-ae6f-890778bdbc4b,6/23/2022 12:00:00 AM,NaN,NaN,EMILY.BARTEE,NaN,NaN,-9.693001e+06,4.546117e+06
3,4,0,UNKNOWN,811 RAVEN AVE,BOWLING GREEN,KY,42101,Warren,BRADD,,...,NaN,696903f3-df9c-4e6a-9fe8-847edd303505,9/8/2021 12:00:00 AM,NaN,NaN,EMILY.BARTEE,NaN,NaN,-9.623290e+06,4.440055e+06
4,5,244318100,Sister Schubert's Homemade Rolls Inc,900 Top Quality Drive,Horse Cave,KY,42749-,Hart,BRADD,"311812, 493120",...,NaN,474ce242-fb81-43c4-9a4c-8c710dcc4d79,6/2/2020 12:00:00 AM,NaN,NaN,EMILY.BARTEE,NaN,NaN,-9.564086e+06,4.461565e+06


In [489]:
#show the first few rows of the DataFrame
traffic_df.head()

,OBJECTID,Facility Name,Facility Street Address,City Name,State,ZIP Code,County Name,Facility Description,Facility SQ Feet,Number of Employees,...,Update Indicator,End Active Date,GUID_Str,Last Edited Date,GlobalID,created_user,created_date,last_edited_user,x,y
0,1,Taylor County Hospital,1700 Old Lebanon Rd,Campbellsville,KY,42718,Taylor,Hospital,NaN,50.0,...,2,NaN,{F9331DB3-A9F7-40D1-8BE4-002E737AB0B6},9/25/2024 12:00:00 AM,cf32d3d3-1070-49db-8d43-fb48a591ca89,NaN,NaN,NaN,-9.499667e+06,4.490203e+06
1,2,FedEx Freight,5501 Burnt Pine Way,Louisville,KY,40228,Jefferson,Logistics,150000.0,90.0,...,2,NaN,{D57C4479-3E9C-4EBB-93F5-0035FBBFAFD9},8/30/2023 12:00:00 AM,9b84814a-525c-4159-a9e4-191ba9ce183f,NaN,NaN,NaN,-9.533609e+06,4.604194e+06
2,3,Cyrus One Data Center,7200 Industrial Rd,Florence,KY,41042,Boone,,NaN,NaN,...,2,NaN,{E23BC34C-CA27-40B7-9901-0066BC29D108},10/11/2024 12:00:00 AM,73337b68-b8f6-4c96-b22c-91a2623dee58,NaN,NaN,NaN,-9.419702e+06,4.718999e+06
3,4,Second Baptist Church,217 W Depot St,Greenville,KY,42345,Muhlenberg,Church,62152.0,NaN,...,2,NaN,{0E2394DC-184F-4D04-81AE-006707B0B8C1},8/24/2023 12:00:00 AM,69501dc3-5517-486f-a753-6891effdecf8,NaN,NaN,NaN,-9.705271e+06,4.468916e+06
4,5,Todd County Central High,US-181 S,Elkton,KY,42220,Todd,School,103698.0,NaN,...,2,NaN,{F2CFDD4D-9FE9-4B1C-984A-007E93F5224E},8/24/2023 12:00:00 AM,73218952-bacb-4fc8-bfb4-7ce783a06fb3,NaN,NaN,NaN,-9.702753e+06,4.410934e+06


In [490]:
#information about the DataFrame
freight_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1971 entries, 0 to 1970
Data columns (total 44 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   OBJECTID                        1971 non-null   int64  
 1   KBIF ID                         1971 non-null   int64  
 2   Facility Name                   1971 non-null   object 
 3   Facility Street Address         1971 non-null   object 
 4   City Name                       1971 non-null   object 
 5   State                           1971 non-null   object 
 6   Zip Code                        1971 non-null   object 
 7   County Name                     1971 non-null   object 
 8   Area Development District       1971 non-null   object 
 9   NAICS Code                      1961 non-null   object 
 10  Building Sq Footage             1971 non-null   int64  
 11  Number of Employees             1971 non-null   int64  
 12  Average Trucks per Day          19

In [491]:
#information about the DataFrame
traffic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4569 entries, 0 to 4568
Data columns (total 41 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   OBJECTID                        4569 non-null   int64  
 1   Facility Name                   4569 non-null   object 
 2   Facility Street Address         4569 non-null   object 
 3   City Name                       4569 non-null   object 
 4   State                           4569 non-null   object 
 5   ZIP Code                        4569 non-null   object 
 6   County Name                     4569 non-null   object 
 7   Facility Description            4569 non-null   object 
 8   Facility SQ Feet                1689 non-null   float64
 9   Number of Employees             796 non-null    float64
 10  Average Trucks per Day          4569 non-null   int64  
 11  Truck Bays                      4569 non-null   int64  
 12  Average Trains per Week         45

In [492]:
#show the summary statistics of the DataFrame
freight_df.describe()

,OBJECTID,KBIF ID,Building Sq Footage,Number of Employees,Average Trucks per Day,Truck Bays,Average Trains Week,Facility Length of Rail,Number of Rail Doors,County Number,...,Route Suffix,Route Label,Update Indicator,End Active Date,created_user,created_date,MP,RT_NUMBER,x,y
count,1971.00000,1.971000e+03,1.971000e+03,1971.000000,1971.000000,1971.000000,1971.000000,1971.000000,1971.000000,1971.000000,...,0.0,0.0,1971.0,0.0,0.0,0.0,0.0,0.0,1.971000e+03,1.971000e+03
mean,986.00000,6.764963e+07,2.389283e+05,165.424658,33.143075,19.493151,3.122273,259.718924,0.254693,50.782344,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,-9.508469e+06,4.587371e+06
std,569.12301,1.624747e+08,4.986740e+05,439.587803,73.669290,34.666201,91.841901,1694.043626,1.892296,32.365878,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1.215144e+05,9.568393e+04
min,1.00000,0.000000e+00,3.000000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,-9.930879e+06,4.373249e+06
25%,493.50000,0.000000e+00,7.662250e+04,0.000000,8.000000,6.000000,0.000000,0.000000,0.000000,24.000000,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,-9.554158e+06,4.534981e+06
50%,986.00000,1.642000e+03,1.418920e+05,59.000000,18.000000,11.000000,0.000000,0.000000,0.000000,56.000000,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,-9.519118e+06,4.600606e+06
75%,1478.50000,7.306500e+03,2.500000e+05,165.000000,39.000000,21.000000,0.000000,0.000000,0.000000,63.000000,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,-9.420047e+06,4.620298e+06
max,1971.00000,6.832346e+08,1.277395e+07,8500.000000,2500.000000,1000.000000,4000.000000,45000.000000,48.000000,120.000000,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,-9.172870e+06,4.738333e+06


In [493]:
#show the summary statistics of the DataFrame
traffic_df.describe()

,OBJECTID,Facility SQ Feet,Number of Employees,Average Trucks per Day,Truck Bays,Average Trains per Week,Number of Rail Doors,Facility Length of Rail,Prior CHIF ID,Completed CHIF ID,...,Route Number,Route Label,RT Unique,Update Indicator,End Active Date,created_user,created_date,last_edited_user,x,y
count,4569.00000,1.689000e+03,796.000000,4569.000000,4569.000000,4569.000000,4569.000000,4569.000000,0.0,0.0,...,0.0,0.0,0.0,4569.0,0.0,0.0,0.0,0.0,4.569000e+03,4.569000e+03
mean,2285.00000,2.168867e+06,951.288945,3.910265,3.151455,0.917706,0.039396,39.228496,NaN,NaN,...,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,-9.485204e+06,4.559648e+06
std,1319.10102,4.812638e+07,11367.274507,20.536708,13.745396,59.182376,0.715556,483.024056,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,1.461996e+05,9.661141e+04
min,1.00000,3.480000e+02,1.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,NaN,NaN,...,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,-9.924762e+06,4.370124e+06
25%,1143.00000,3.900000e+04,35.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,...,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,-9.565238e+06,4.480117e+06
50%,2285.00000,1.030000e+05,78.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,...,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,-9.444717e+06,4.559518e+06
75%,3427.00000,2.500000e+05,175.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,...,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,-9.402970e+06,4.612122e+06
max,4569.00000,1.923530e+09,227800.000000,520.000000,260.000000,4000.000000,35.000000,15000.000000,NaN,NaN,...,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,-9.136321e+06,4.738441e+06


In [494]:
#number of rows and columns
print("Freight DF shape:", freight_df.shape)
print("Traffic DF shape:", traffic_df.shape)

Freight DF shape: (1971, 44)
Traffic DF shape: (4569, 41)


In [495]:
# shows the data types and column names of the DataFrame
print(freight_df.dtypes)


OBJECTID                            int64
KBIF ID                             int64
Facility Name                      object
Facility Street Address            object
City Name                          object
State                              object
Zip Code                           object
County Name                        object
Area Development District          object
NAICS Code                         object
Building Sq Footage                 int64
Number of Employees                 int64
Average Trucks per Day              int64
Truck Bays                          int64
Average Trains Week                 int64
Facility Length of Rail             int64
Number of Rail Doors                int64
Prior CHAF ID                      object
Completed CHAF ID                  object
Existing CHAF ID                   object
Intermodal Connector Needs         object
Comments                           object
Meets Criteria                     object
Area Development District Name    

In [496]:
# shows the data types and column names of the DataFrame
print(traffic_df.dtypes)

OBJECTID                            int64
Facility Name                      object
Facility Street Address            object
City Name                          object
State                              object
ZIP Code                           object
County Name                        object
Facility Description               object
Facility SQ Feet                  float64
Number of Employees               float64
Average Trucks per Day              int64
Truck Bays                          int64
Average Trains per Week             int64
Number of Rail Doors                int64
Facility Length of Rail             int64
Web URL                            object
Comments                           object
Prior CHIF ID                     float64
Completed CHIF ID                 float64
Existing CHAF ID                  float64
District Number                     int64
District Name                      object
Area Development District          object
Area Development District Name    

In [497]:
# Check for missing values
print("Missing values in Freight DF:")
print(freight_df.isnull().sum())


Missing values in Freight DF:
OBJECTID                             0
KBIF ID                              0
Facility Name                        0
Facility Street Address              0
City Name                            0
State                                0
Zip Code                             0
County Name                          0
Area Development District            0
NAICS Code                          10
Building Sq Footage                  0
Number of Employees                  0
Average Trucks per Day               0
Truck Bays                           0
Average Trains Week                  0
Facility Length of Rail              0
Number of Rail Doors                 0
Prior CHAF ID                      397
Completed CHAF ID                    0
Existing CHAF ID                     0
Intermodal Connector Needs           0
Comments                            19
Meets Criteria                       0
Area Development District Name       0
County Number                     

In [498]:
# Check for missing values
print("Missing values in Traffic DF:")
print(traffic_df.isnull().sum())

Missing values in Traffic DF:
OBJECTID                             0
Facility Name                        0
Facility Street Address              0
City Name                            0
State                                0
ZIP Code                             0
County Name                          0
Facility Description                 0
Facility SQ Feet                  2880
Number of Employees               3773
Average Trucks per Day               0
Truck Bays                           0
Average Trains per Week              0
Number of Rail Doors                 0
Facility Length of Rail              0
Web URL                              0
Comments                             0
Prior CHIF ID                     4569
Completed CHIF ID                 4569
Existing CHAF ID                  4569
District Number                      0
District Name                        0
Area Development District            0
Area Development District Name       0
County Number                     

In [499]:
# convert all column names to lowercase to make easier to work with
traffic_df.columns = traffic_df.columns.str.lower()
# replace spaces with underscores to make easier to work with
traffic_df.columns = traffic_df.columns.str.replace(' ', '_')

In [500]:
# A list of columns to remove from the traffic_df DataFrame
remove_columns_traffic_df = [
    'last_edited_user',
    'created_date',
    'created_user',
    'last_edited_date',
    'end_active_date',
    'update_indicator',
    'rt_unique',
    'route_label',
    'route_number',
    'route_prefix',
    'road_name',
    'milepoint',
    'existing_chaf_id',
    'completed_chif_id',
    'prior_chif_id',
    'comments',
    'facility_length_of_rail',
    'number_of_rail_doors',
    'average_trains_per_week',
    'number_of_employees',
    'facility_sq_feet',
    'objectid',
    'guid_str',
    'globalid'


]
#drops columns listed in reomove_columns_traffic_df from traffic_df
traffic_df_cleaned = traffic_df.drop(columns=remove_columns_traffic_df)

In [501]:
# I did this to check what were in the columns when deciding which to drop used the same line of code and just changed the column name
print(traffic_df['zip_code'].head())

0    42718
1    40228
2    41042
3    42345
4    42220
Name: zip_code, dtype: object


In [502]:
# Check for missing values
print("Missing values in Freight DF:")
print(freight_df.isnull().sum())

Missing values in Freight DF:
OBJECTID                             0
KBIF ID                              0
Facility Name                        0
Facility Street Address              0
City Name                            0
State                                0
Zip Code                             0
County Name                          0
Area Development District            0
NAICS Code                          10
Building Sq Footage                  0
Number of Employees                  0
Average Trucks per Day               0
Truck Bays                           0
Average Trains Week                  0
Facility Length of Rail              0
Number of Rail Doors                 0
Prior CHAF ID                      397
Completed CHAF ID                    0
Existing CHAF ID                     0
Intermodal Connector Needs           0
Comments                            19
Meets Criteria                       0
Area Development District Name       0
County Number                     

In [503]:
# convert all column names to lowercase to make easier to work with
freight_df.columns = freight_df.columns.str.lower()
# replace spaces with underscores to make easier to work with
freight_df.columns = freight_df.columns.str.replace(' ', '_')
# renamed zip_code_ to zip_code to match traffic_df
freight_df.rename(columns={'zip_code_': 'zip_code'}, inplace=True)


In [504]:
# A list of columns to remove from the freight_df DataFrame
remove_columns_freight_df = [
    'rt_number',
    'mp',
    'created_date',
    'last_edited_user',
    'created_user',
    'last_edited_date',
    'end_active_date',
    'update_indicator',
    'route_label',
    'route_suffix',
    'route_prefix',
    'road_name',
    'rt_unique',
    'comments',
    'intermodal_connector_needs',
    'existing_chaf_id',
    'completed_chaf_id',
    'prior_chaf_id',
    'number_of_rail_doors',
    'facility_length_of_rail',
    'average_trains_week',
    'objectid',
    'kbif_id',
    'guid_str',
    'meets_criteria',
    'globalid',
    ]
#drops columns listed in reomove_columns_freight_df from freight_df
freight_df_cleaned = freight_df.drop(columns=remove_columns_freight_df)

In [505]:
# I did this to check what were in the columns when deciding which to drop used the same line of code and just changed the column name
print(freight_df['meets_criteria'].head())

0    YES
1    YES
2    YES
3    YES
4    YES
Name: meets_criteria, dtype: object


In [506]:
# printing the column names of the cleaned traffic_df to see what is left and to prepare for merging
print(traffic_df_cleaned.columns)

Index(['facility_name', 'facility_street_address', 'city_name', 'state',
       'zip_code', 'county_name', 'facility_description',
       'average_trucks_per_day', 'truck_bays', 'web_url', 'district_number',
       'district_name', 'area_development_district',
       'area_development_district_name', 'county_number', 'x', 'y'],
      dtype='object')


In [507]:
# printing the column names of the cleaned freight_df to see what is left and to prepare for merging
print(freight_df_cleaned.columns)

Index(['facility_name', 'facility_street_address', 'city_name', 'state',
       'zip_code', 'county_name', 'area_development_district', 'naics_code',
       'building_sq_footage', 'number_of_employees', 'average_trucks_per_day',
       'truck_bays', 'area_development_district_name', 'county_number',
       'district_number', 'district_name', 'x', 'y'],
      dtype='object')


In [508]:
# this function normalizes all string columns in a dataframe
def normalize_all_strings(df):
    for col in df.select_dtypes(include='object').columns:
        df[col] = (
            df[col]
            .fillna('')
            .str.lower()
            .str.strip()
            # drops all punctuiation
            .str.replace(r'[^\w\s]', '', regex=True)
            # colapses multiple spaces into a single space
            .str.replace(r'\s+', ' ', regex=True)
            .str.replace('apt', '', regex=False)
            .str.replace('suite', '', regex=False)
            .str.replace('#', '', regex=False)
            .str.replace(r'\d+[a-zA-Z]?$', '', regex=True)
            .str.replace('unit', '', regex=False)
            .str.replace('building', '', regex=False)
            .str.replace('floor', '', regex=False)
            .str.replace('room', '', regex=False)
            .str.replace('dept', '', regex=False)
            .str.replace('division', '', regex=False)
            .str.replace('office', '', regex=False)
            .str.replace('north', 'n', regex=False)
            .str.replace('south', 's', regex=False)
            .str.replace('east', 'e', regex=False)
            .str.replace('west', 'w', regex=False)
            .str.replace(r'\bnorth\b', 'n', regex=True)
            .str.replace(r'\bsouth\b', 's', regex=True)
            .str.replace(r'\beast\b', 'e', regex=True)
            .str.replace(r'\bwest\b', 'w', regex=True)
            .str.replace(r'\bnortheast\b', 'ne', regex=True)
            .str.replace(r'\bsouthwest\b', 'sw', regex=True)
            .str.replace('circle', 'cir', regex=False)
            .str.replace('place', 'pl', regex=False)
            .str.replace('terrace', 'ter', regex=False)
            .str.replace('parkway', 'pkwy', regex=False)
            .str.replace('expressway', 'expy', regex=False)
            .str.replace('highway', 'hwy', regex=False)
            # removes metatags and other unnecessary text
            .str.replace(r'\s*\(.*?\)', '', regex=True)
            .str.replace(r'\s*-\s*.*$', '', regex=True)
            .str.replace(r'\b(llc|inc|corp|co|ltd)\b', '', regex=True)
            .str.replace('distribution center', 'dc', regex=False)
            .str.replace('fulfillment center', 'fc', regex=False)
            .str.replace('warehouse', 'whse', regex=False)
            .str.replace('street', 'st', regex=False)
            .str.replace('avenue', 'ave', regex=False)
            .str.replace('drive', 'dr', regex=False)
            .str.replace('road', 'rd', regex=False)
            .str.replace('boulevard', 'blvd', regex=False)
            .str.replace('lane', 'ln', regex=False)
            .str.replace('court', 'ct', regex=False)
            .str.replace(r'\s+', ' ', regex=True)
            .str.strip()
        )

        if col == 'zip_code':
            df[col] = (
                df[col]
                .str.extract(r'(\d{5})')[0]
                .fillna('00000')
                .astype(str)
                .str.zfill(5)
            )

    return df


In [509]:
# uses the normalize strings function and assigns the cleaned dataframes to new variables
freight_df_cleaned = normalize_all_strings(freight_df_cleaned)
traffic_df_cleaned = normalize_all_strings(traffic_df_cleaned)

In [510]:
# this shows zip codes that are in freight_df but not in traffic_df
freight_zips = set(freight_df_cleaned['zip_code'])
traffic_zips = set(traffic_df_cleaned['zip_code'])
print(freight_zips - traffic_zips) 

set()


In [511]:
# joins the two DataFrames on the zip_code, county_number, and street address columns freight_df merged columns will end in _x and traffic_df will end in _y  
# the keys used (zip_code and county_number) are merged into one and will not end in _x or _y
# this is a inner join so only rows with matching keys in both DataFrames will be included
merged_df = pd.merge(freight_df_cleaned, traffic_df_cleaned, on=['zip_code', 'facility_street_address'], how='inner')

In [512]:
# used this to check the shape of the merged DataFrame to see which keys would be best to merge on
merged_df.shape

(823, 33)

In [513]:
merged_df.columns

Index(['facility_name_x', 'facility_street_address', 'city_name_x', 'state_x',
       'zip_code', 'county_name_x', 'area_development_district_x',
       'naics_code', 'building_sq_footage', 'number_of_employees',
       'average_trucks_per_day_x', 'truck_bays_x',
       'area_development_district_name_x', 'county_number_x',
       'district_number_x', 'district_name_x', 'x_x', 'y_x', 'facility_name_y',
       'city_name_y', 'state_y', 'county_name_y', 'facility_description',
       'average_trucks_per_day_y', 'truck_bays_y', 'web_url',
       'district_number_y', 'district_name_y', 'area_development_district_y',
       'area_development_district_name_y', 'county_number_y', 'x_y', 'y_y'],
      dtype='object')

In [514]:
# shows the first few rows of the merged DataFrame to see what it looks like
merged_df.head(10)

,facility_name_x,facility_street_address,city_name_x,state_x,zip_code,county_name_x,area_development_district_x,naics_code,building_sq_footage,number_of_employees,...,average_trucks_per_day_y,truck_bays_y,web_url,district_number_y,district_name_y,area_development_district_y,area_development_district_name_y,county_number_y,x_y,y_y
0,corning incorporated,680 e st,harrodsburg,ky,00000,mercer,bgadd,,198500,400,...,0,0,,7,lexington,bgadd,bluegrass add,84,-9.443209e+06,4.545300e+06
1,omico plastics,2025 ragu dr,owensboro,ky,00000,daviess,gradd,,196000,67,...,12,3,wwwomicoplasticscom,2,madisonville,gradd,green river add,30,-9.693000e+06,4.546118e+06
2,hobart,58 corporate dr,danville,ky,00000,boyle,bgadd,,90000,114,...,0,0,,7,lexington,bgadd,bluegrass add,11,-9.441789e+06,4.526259e+06
3,bap,1261 aviation blvd,hebron,ky,00000,boone,nkadd,,112000,35,...,36,36,,6,covington,nkadd,nern kentucky add,8,-9.426281e+06,4.733320e+06
4,container packaging,1001 glengarry dr,fairdale,ky,00000,jefferson,kipda,,168000,0,...,40,20,,5,louisville,kipda,kipda add,56,-9.545112e+06,4.597124e+06
5,mueller roofing,4301 boron ave,covington,ky,00000,kenton,nkadd,,138000,5,...,3,3,,6,covington,nkadd,nern kentucky add,59,-9.407878e+06,4.727720e+06
6,graham packaging plastic products,8283 dixie hwy,florence,ky,00000,boone,nkadd,,155000,54,...,10,10,,6,covington,nkadd,nern kentucky add,8,-9.419147e+06,4.717597e+06
7,flextronics america,4400 commerce crossings dr,louisville,ky,00000,jefferson,kipda,,500000,326,...,56,28,flexcom,5,louisville,kipda,kipda add,56,-9.538683e+06,4.592510e+06
8,cvg2 amazoncomkydcllc,1600 worldwide blvd,hebron,ky,00000,boone,nkadd,423990 443142 454111 454113 493110,542960,2000,...,106,106,,6,covington,nkadd,nern kentucky add,8,-9.431644e+06,4.734134e+06
9,honeywell integrated,201 richard knock hwy,walton,ky,00000,kenton,nkadd,,900000,0,...,150,150,,6,covington,nkadd,nern kentucky add,59,-9.417133e+06,4.701791e+06


In [515]:
merged_df.tail(10)

,facility_name_x,facility_street_address,city_name_x,state_x,zip_code,county_name_x,area_development_district_x,naics_code,building_sq_footage,number_of_employees,...,average_trucks_per_day_y,truck_bays_y,web_url,district_number_y,district_name_y,area_development_district_y,area_development_district_name_y,county_number_y,x_y,y_y
813,florida tile,1211 alton rd,lawrenceburg,ky,00000,anderson,bgadd,,428818,198,...,0,0,,7,lexington,bgadd,bluegrass add,3,-9.454027e+06,4.589675e+06
814,clark distributing,200 commerce dr,madisonville,ky,00000,hopkins,peadd,,528788,0,...,0,0,,2,madisonville,peadd,pennyrile add,54,-9.738969e+06,4.487775e+06
815,eagle steel products,3420 collins ln,louisville,ky,00000,jefferson,kipda,,110000,30,...,20,6,wwweaglesteelproductscom,5,louisville,kipda,kipda add,56,-9.520513e+06,4.621457e+06
816,watco logistics,6040 shepherdsville rd,louisville,ky,00000,jefferson,kipda,,140000,0,...,40,20,rscscom,5,louisville,kipda,kipda add,56,-9.535825e+06,4.602208e+06
817,camco chemical,8150 holton dr,independence,ky,00000,kenton,nkadd,325611,230000,200,...,12,12,,6,covington,nkadd,nern kentucky add,59,-9.418839e+06,4.717945e+06
818,pratt target container,1901 international way,hebron,ky,00000,boone,nkadd,,220000,57,...,21,21,,6,covington,nkadd,nern kentucky add,8,-9.428862e+06,4.733985e+06
819,winchester coatings,1600 fortune dr,winchester,ky,00000,clark,bgadd,,69923,85,...,0,0,wwwwincoatnet,7,lexington,bgadd,bluegrass add,25,-9.370947e+06,4.582447e+06
820,us playing card company,300 gap way,erlanger,ky,00000,boone,nkadd,,520000,0,...,44,44,,6,covington,nkadd,nern kentucky add,8,-9.421158e+06,4.728321e+06
821,galerie,3380 langley dr,hebron,ky,00000,boone,nkadd,424450 493110,277000,110,...,23,23,,6,covington,nkadd,nern kentucky add,8,-9.429388e+06,4.730566e+06
822,signature hardware,2700 crescent springs pk,erlanger,ky,00000,kenton,nkadd,,304000,211,...,24,24,,6,covington,nkadd,nern kentucky add,59,-9.416099e+06,4.727910e+06


In [516]:
# this script counts the number of rows where both facility_name_x and facility_name_y are 'UNKNOWN'
# I converted entries to uppercase then checked if either facility_name_x or facility_name_y is 'UNKNOWN' if so it would == True
check_unknowns = (merged_df['facility_name_x'] == 'unknown') | (merged_df['facility_name_y'] == 'unknown')
total_unknowns = check_unknowns.sum()
print(total_unknowns)

3


In [517]:
merged_df.isnull().sum()

facility_name_x                     0
facility_street_address             0
city_name_x                         0
state_x                             0
zip_code                            0
county_name_x                       0
area_development_district_x         0
naics_code                          0
building_sq_footage                 0
number_of_employees                 0
average_trucks_per_day_x            0
truck_bays_x                        0
area_development_district_name_x    0
county_number_x                     0
district_number_x                   0
district_name_x                     0
x_x                                 0
y_x                                 0
facility_name_y                     0
city_name_y                         0
state_y                             0
county_name_y                       0
facility_description                0
average_trucks_per_day_y            0
truck_bays_y                        0
web_url                             0
district_num

In [518]:
# created a temporary dadtaframe to work instead of the original merged_df
temp_df = merged_df.copy()
# creates 2 new columns with the facility names cleaned up
temp_df['facility_name_x_clean'] = temp_df['facility_name_x']
temp_df['facility_name_y_clean'] = temp_df['facility_name_y']



In [519]:
# checks if the cleaned facility names are the same
same_facility = temp_df['facility_name_x_clean'] == temp_df['facility_name_y_clean']

print(same_facility.sum())

419


In [520]:
# prints the first few rows of the temporary DataFrame where the facility names are the same
temp_df[same_facility].head()

,facility_name_x,facility_street_address,city_name_x,state_x,zip_code,county_name_x,area_development_district_x,naics_code,building_sq_footage,number_of_employees,...,web_url,district_number_y,district_name_y,area_development_district_y,area_development_district_name_y,county_number_y,x_y,y_y,facility_name_x_clean,facility_name_y_clean
1,omico plastics,2025 ragu dr,owensboro,ky,00000,daviess,gradd,,196000,67,...,wwwomicoplasticscom,2,madisonville,gradd,green river add,30,-9.693000e+06,4.546118e+06,omico plastics,omico plastics
2,hobart,58 corporate dr,danville,ky,00000,boyle,bgadd,,90000,114,...,,7,lexington,bgadd,bluegrass add,11,-9.441789e+06,4.526259e+06,hobart,hobart
4,container packaging,1001 glengarry dr,fairdale,ky,00000,jefferson,kipda,,168000,0,...,,5,louisville,kipda,kipda add,56,-9.545112e+06,4.597124e+06,container packaging,container packaging
5,mueller roofing,4301 boron ave,covington,ky,00000,kenton,nkadd,,138000,5,...,,6,covington,nkadd,nern kentucky add,59,-9.407878e+06,4.727720e+06,mueller roofing,mueller roofing
6,graham packaging plastic products,8283 dixie hwy,florence,ky,00000,boone,nkadd,,155000,54,...,,6,covington,nkadd,nern kentucky add,8,-9.419147e+06,4.717597e+06,graham packaging plastic products,graham packaging plastic products


In [521]:
merged_df_copy = merged_df.copy()

In [522]:
# this is a list called group_cols that will be used to group the DataFrame
group_cols = [
    'facility_street_address',
    'zip_code',
    'county_name_x',
    'county_name_y',
    'naics_code'
]

In [523]:
# This groups the Dataframe by the specified columns and aggregates the data
deduped_df = merged_df_copy.groupby(group_cols).agg({
    
# this aggregates the data by taking the maximum value for the specified columns
    'average_trucks_per_day_x': 'max',
    'average_trucks_per_day_y': 'max',
    'truck_bays_x': 'max',
    'truck_bays_y': 'max',

# this averages the x and y coordinates
    'x_x': 'mean',
    'y_x': 'mean',
    'x_y': 'mean',
    'y_y': 'mean',

# this takes the first value for building sq footage and the maximum for number of employees
    'building_sq_footage': 'first',
    'number_of_employees': 'max',

# this takes the first value for the district name and area development district
    'district_name_x': 'first',
    'district_name_y': 'first',
    'area_development_district_x': 'first',
    'area_development_district_y': 'first',

# this combines the web URLs into one line prevents duplicates and dropes any NaN values
# lambda is used to create a custom aggregation function ', ' adds a comma and space between URLs join puts the urls on the same line
# sorted puts the URLs in alphabetical order set keeps only unique URLs dropna removes any NaN values
    'web_url': lambda x: ', '.join(sorted(set(x.dropna()))),

# this combines the facility names into one line prevents duplicates and dropes any NaN values
    'facility_name_x': lambda x: ', '.join(sorted(set(x.dropna()))),
    'facility_name_y': lambda x: ', '.join(sorted(set(x.dropna()))),

# this moves the groupby coulmns back to the original place and replaces index with a interger
}).reset_index()

In [524]:
deduped_df.head()

,facility_street_address,zip_code,county_name_x,county_name_y,naics_code,average_trucks_per_day_x,average_trucks_per_day_y,truck_bays_x,truck_bays_y,x_x,...,y_y,building_sq_footage,number_of_employees,district_name_x,district_name_y,area_development_district_x,area_development_district_y,web_url,facility_name_x,facility_name_y
0,,00000,boone,bell,,95,0,48,0,-9.429933e+06,...,4.401354e+06,528000,100,covington,manchester,nkadd,cvadd,httpsboonesridgecom,win it america,boons ridge
1,,00000,boone,boone,,95,25,48,25,-9.429933e+06,...,4.717796e+06,528000,100,covington,covington,nkadd,nkadd,,win it america,beckman coulter
2,,00000,boone,breathitt,,95,0,48,0,-9.429933e+06,...,4.513728e+06,528000,100,covington,jackson,nkadd,kradd,,win it america,"breathitt county atc, breathitt county high sc..."
3,,00000,boone,grayson,,95,0,48,0,-9.429933e+06,...,4.505889e+06,528000,100,covington,elizabethtown,nkadd,ltadd,,win it america,general services exit
4,,00000,boone,jefferson,,95,0,48,0,-9.429933e+06,...,4.608566e+06,528000,100,covington,louisville,nkadd,kipda,,win it america,"buechel rail yard, ge rail yard, kyard rail ya..."


In [525]:
# this shows the rows and columns of the deduped DataFrame
deduped_df.shape

(694, 22)

In [526]:
# this shows the columns of the deduped DataFrame
deduped_df.columns

Index(['facility_street_address', 'zip_code', 'county_name_x', 'county_name_y',
       'naics_code', 'average_trucks_per_day_x', 'average_trucks_per_day_y',
       'truck_bays_x', 'truck_bays_y', 'x_x', 'y_x', 'x_y', 'y_y',
       'building_sq_footage', 'number_of_employees', 'district_name_x',
       'district_name_y', 'area_development_district_x',
       'area_development_district_y', 'web_url', 'facility_name_x',
       'facility_name_y'],
      dtype='object')

In [527]:
deduped_df.head()

,facility_street_address,zip_code,county_name_x,county_name_y,naics_code,average_trucks_per_day_x,average_trucks_per_day_y,truck_bays_x,truck_bays_y,x_x,...,y_y,building_sq_footage,number_of_employees,district_name_x,district_name_y,area_development_district_x,area_development_district_y,web_url,facility_name_x,facility_name_y
0,,00000,boone,bell,,95,0,48,0,-9.429933e+06,...,4.401354e+06,528000,100,covington,manchester,nkadd,cvadd,httpsboonesridgecom,win it america,boons ridge
1,,00000,boone,boone,,95,25,48,25,-9.429933e+06,...,4.717796e+06,528000,100,covington,covington,nkadd,nkadd,,win it america,beckman coulter
2,,00000,boone,breathitt,,95,0,48,0,-9.429933e+06,...,4.513728e+06,528000,100,covington,jackson,nkadd,kradd,,win it america,"breathitt county atc, breathitt county high sc..."
3,,00000,boone,grayson,,95,0,48,0,-9.429933e+06,...,4.505889e+06,528000,100,covington,elizabethtown,nkadd,ltadd,,win it america,general services exit
4,,00000,boone,jefferson,,95,0,48,0,-9.429933e+06,...,4.608566e+06,528000,100,covington,louisville,nkadd,kipda,,win it america,"buechel rail yard, ge rail yard, kyard rail ya..."


In [528]:
def compare_columns(df, col1, col2):
# this cleans the columns by stripping whitespace and converting to uppercase
    val1 = df[col1].astype(str).str.strip().str.upper()
    val2 = df[col2].astype(str).str.strip().str.upper()
# this compares the two columns and returns true or false for each row
    match = val1 == val2

# this counts the number of matches and mismatches
    num_matches = match.sum()
    num_mismatches = len(df) - num_matches

# this prints the results
    print(f"Total matches between '{col1}' and '{col2}': {num_matches}")
    print(f"Total mismatches: {num_mismatches}\n")
# this prints the mismatched rows if there are any
    if num_mismatches > 0:
        print("Mismatched Rows:")
        print(df.loc[~match, [col1, col2]])


In [529]:
compare_columns(deduped_df, 'facility_name_x', 'facility_name_y')

Total matches between 'facility_name_x' and 'facility_name_y': 385
Total mismatches: 309

Mismatched Rows:
              facility_name_x  \
0              win it america   
1              win it america   
2              win it america   
3              win it america   
4              win it america   
..                        ...   
689  ghent generating station   
690  ghent generating station   
691  ghent generating station   
692  ghent generating station   
693  ghent generating station   

                                       facility_name_y  
0                                          boons ridge  
1                                      beckman coulter  
2    breathitt county atc, breathitt county high sc...  
3                                general services exit  
4    buechel rail yard, ge rail yard, kyard rail ya...  
..                                                 ...  
689                                ak steel coke plant  
690                                 nern

In [531]:
merged_df[['x_x', 'x_y', 'y_x', 'y_y']]

,x_x,x_y,y_x,y_y
0,-9.443214e+06,-9.443209e+06,4.545184e+06,4.545300e+06
1,-9.693001e+06,-9.693000e+06,4.546117e+06,4.546118e+06
2,-9.441758e+06,-9.441789e+06,4.526258e+06,4.526259e+06
3,-9.426292e+06,-9.426281e+06,4.733377e+06,4.733320e+06
4,-9.545117e+06,-9.545112e+06,4.597131e+06,4.597124e+06
...,...,...,...,...
818,-9.428877e+06,-9.428862e+06,4.734039e+06,4.733985e+06
819,-9.370975e+06,-9.370947e+06,4.582510e+06,4.582447e+06
820,-9.421158e+06,-9.421158e+06,4.728322e+06,4.728321e+06
821,-9.429389e+06,-9.429388e+06,4.730564e+06,4.730566e+06
